# 👁️ Gaze Estimation with UniFace

<div style="display:flex; flex-wrap:wrap; align-items:center;">
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pepy.tech/projects/uniface"><img alt="PyPI Downloads" src="https://static.pepy.tech/badge/uniface"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pypi.org/project/uniface/"><img alt="PyPI Version" src="https://img.shields.io/pypi/v/uniface.svg"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://opensource.org/licenses/MIT"><img alt="License" src="https://img.shields.io/badge/License-MIT-blue.svg"></a>
  <a style="margin-bottom:6px;" href="https://github.com/yakhyo/uniface"><img alt="GitHub Stars" src="https://img.shields.io/github/stars/yakhyo/uniface.svg?style=social"></a>
</div>

**UniFace** is a lightweight, production-ready, all-in-one face analysis library built on ONNX Runtime.

🔗 **GitHub**: [github.com/yakhyo/uniface](https://github.com/yakhyo/uniface) | 📚 **Docs**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)

---

## 📖 Overview

This notebook demonstrates **gaze estimation** - predicting where a person is looking:

- ✅ Estimate gaze direction (pitch and yaw angles)
- ✅ Visualize gaze with direction arrows
- ✅ Process multiple face images

## 1️⃣ Installation

In [ ]:
%pip install -q uniface

import os
import urllib.request

os.makedirs('assets/test_images', exist_ok=True)

BASE_URL = "https://raw.githubusercontent.com/yakhyo/uniface/main/assets"
images = ["test_images/image0.jpg", "test_images/image1.jpg", "test_images/image2.jpg",
          "test_images/image3.jpg", "test_images/image4.jpg"]

for img in images:
    if not os.path.exists(f'assets/{img}'):
        urllib.request.urlretrieve(f"{BASE_URL}/{img}", f"assets/{img}")
        print(f"✓ Downloaded {img}")

## 2️⃣ Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import uniface
from uniface.detection import RetinaFace
from uniface.gaze import MobileGaze
from uniface.visualization import draw_gaze

print(f"UniFace version: {uniface.__version__}")

## 3️⃣ Initialize Models

In [ ]:
detector = RetinaFace(confidence_threshold=0.5)
gaze_estimator = MobileGaze()

## 4️⃣ Process All Test Images

We'll detect faces, estimate gaze, and visualize results.

In [ ]:
test_images_dir = Path('assets/test_images')
test_images = sorted(test_images_dir.glob('*.jpg'))

original_images = []
processed_images = []

for image_path in test_images:
    print(f"Processing: {image_path.name}")

    image = cv2.imread(str(image_path))
    original = image.copy()

    faces = detector.detect(image)
    print(f'   ✓ Detected {len(faces)} face(s)')

    for i, face in enumerate(faces):
        x1, y1, x2, y2 = map(int, face.bbox[:4])
        face_crop = image[y1:y2, x1:x2]

        if face_crop.size > 0:
            gaze = gaze_estimator.estimate(face_crop)
            pitch_deg = np.degrees(gaze.pitch)
            yaw_deg = np.degrees(gaze.yaw)

            print(f'   → Face {i+1}: pitch={pitch_deg:.1f}°, yaw={yaw_deg:.1f}°')
            draw_gaze(image, face.bbox, gaze.pitch, gaze.yaw, draw_angles=False)

    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    processed_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    original_images.append(original_rgb)
    processed_images.append(processed_rgb)

print(f"\n✓ Processed {len(test_images)} images")

## 5️⃣ Visualize Results

**Row 1**: Original images  
**Row 2**: Images with gaze direction arrows

In [ ]:
num_images = len(original_images)

fig, axes = plt.subplots(2, num_images, figsize=(4*num_images, 8))

if num_images == 1:
    axes = axes.reshape(2, 1)

for i, img in enumerate(original_images):
    axes[0, i].imshow(img)
    axes[0, i].set_title(f'Original {i+1}', fontsize=12)
    axes[0, i].axis('off')

for i, img in enumerate(processed_images):
    axes[1, i].imshow(img)
    axes[1, i].set_title(f'Gaze {i+1}', fontsize=12)
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

---

## 📝 Summary

| Feature | Description |
|---------|-------------|
| **Input** | Face crop from face detection |
| **Output** | `GazeResult` with `pitch` and `yaw` (radians) |
| **Pitch** | Up/down angle (+ = looking up) |
| **Yaw** | Left/right angle (+ = looking right) |
| **Model** | MobileGaze (trained on Gaze360 dataset) |
| **Accuracy** | MAE ~11-13 degrees |

### Tips for Best Results

- ✅ Ensure faces are clearly visible and well-lit
- ✅ Works best with frontal to semi-profile faces
- ⚠️ Accuracy may vary with extreme head poses
- ⚠️ Occlusions (sunglasses, hair) may affect results

---

## 🔗 Explore More Notebooks

| Notebook | Description |
|----------|-------------|
| [01_face_detection](./01_face_detection.ipynb) | Detect faces with bounding boxes |
| [02_face_alignment](./02_face_alignment.ipynb) | Align faces for recognition |
| [03_face_verification](./03_face_verification.ipynb) | Compare two faces |
| [04_face_search](./04_face_search.ipynb) | Find a person in a crowd |
| [05_face_analyzer](./05_face_analyzer.ipynb) | Age & gender prediction |
| [06_face_parsing](./06_face_parsing.ipynb) | Semantic segmentation |
| [07_face_anonymization](./07_face_anonymization.ipynb) | Privacy protection |

📚 **Full Documentation**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)